# Summary
We create images from the Google Open Buildings (2022) and export them to Google Cloud Storage. From there we can create tiles locally.

### Future
- Do the same for the Google Open Buildings (2021)
- Incorporate Google Dynamic World once the rock filter works better

# Setting up the environment

In [ ]:
# Import and/or install libraries

import subprocess, os

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Getting started

We divided Africa into blocks and will run each block manually as a start.

In [ ]:
import json

# 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 18, 19, 20, 21, 23, 24, 25

id = 16

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for feature in json_data['features']:
      if feature['properties']['id'] == id:
        feature['properties']['style'] = {}
        blocks.append(feature)

block = geemap.geojson_to_ee(blocks[0])


In [ ]:
# AOI

aoi = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              28.06574851376746,
              -22.324553464205877
            ],
            [
              28.06574851376746,
              -26.01523517180692
            ],
            [
              33.733773120836446,
              -26.01523517180692
            ],
            [
              33.733773120836446,
              -22.324553464205877
            ],
            [
              28.06574851376746,
              -22.324553464205877
            ]
          ]
        ],
      })

# Prepare layers

First, we experiment with 2022 for our AOI.

In [ ]:
year = 2022
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)

# Google Dynamic World
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.06).selfMask()
best_people = people.mask(certainty_mask).eq(1).selfMask()
# people_c = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').mask(people)

# Mask rocks

In [ ]:
# Import rocks
with open("/content/drive/MyDrive/data/rocks.geojson") as f:
    json_data = json.load(f)
rocks = geemap.geojson_to_ee(json_data)

# Get the AOI from the rocks
aoi = rocks.geometry().bounds()

# Convert built to vectors
built_polygons = best_people.reduceToVectors(
  geometry=aoi,
  crs=best_people.projection(),
  scale=20,
  geometryType='polygon',
  eightConnected=False,
  labelProperty='label',
  maxPixels=137117000,
);

# Convert rocks to raster
img_rocks = ee.Image().paint(**{
  'featureCollection': rocks,
  'color': 1,
})


# Sample
poly_val = img_rocks.reduceRegions(
    collection=built_polygons,
    reducer=ee.Reducer.sum(),
    scale=10)
img_people = poly_val.reduceToImage(
    properties = ['sum'],
    reducer = ee.Reducer.max()
).gt(0).selfMask()

# Choose biggest between rock polygons and people
img_samples = img_rocks.unmask(0).add(img_people.unmask()).gt(0).selfMask()


# Convert raster to polygons
polygons = img_samples.reduceToVectors(
  geometry=aoi,
  crs=best_people.projection(),
  scale=10,
  geometryType='polygon',
  eightConnected=False,
  labelProperty='label',
  maxPixels=568459950,
);



In [ ]:
# Export cleaned up rock mask

exportConfig = {
    'collection': polygons,
    'description': 'built_rock_mask',
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': 'vector/built_rock_mask',
    'fileFormat': 'GeoJSON',
}

task = ee.batch.Export.table.toCloudStorage(**exportConfig)
task.start()

# Google Open Buildings and combine

Rasterise Google's Open Buildings

In [ ]:
# Google Open Buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v2/polygons').filter('confidence >= 0.70');

buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

In [ ]:
# Join with other layers
built = best_people.unmask(0).add(buildings_raster.unmask(0)).gt(0).selfMask()


# Export layers


In [ ]:
# Set configurations for the export

image_name = 'built2022_' + str(id)
fileNamePrefix = 'COGS/built/built2022/' + image_name


exportConfig = {
    'image': buildings_raster,
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 30,
    'maxPixels': 3147395000,
    'region': block,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

# Map

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Map.addLayer(people, {'min':0, 'max':1, 'palette':['white','blue']}, 'people')
# Map.addLayer(best_people, {'min':0, 'max':1, 'palette':['white','red']}, 'best_people')

# Map.addLayer(buildings, {'color': 'red'}, 'Buildings confidence >= 0.70');
# Map.addLayer(buildings_raster, {'min':0, 'max':1, 'palette':['white','red']}, 'buildings_raster')
# Map.addLayer(gdw, {'min':0, 'max':8, 'palette':['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1']}, 'GDW')
# Map.addLayer(people_c, {'min':0, 'max':0.1, 'palette':['white','blue']}, 'people_c')
# Map.addLayer(poly_val, {}, 'poly_val')
# Map.addLayer(img_rocks.unmask(0), {}, 'img_rocks')
# Map.addLayer(img_people.unmask(0), {}, 'img_people')

Map.addLayer(polygons, {}, 'polygons')
# Map.addLayer(rocks, {}, 'rocks')

# Map.addLayer(built, {}, 'built')

Map.setCenter(31.944, -25.087, 12)
Map
